In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
xformers = "xformers==0.0.28.post3"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton
import torch
torch.__version__
# 安装 ollama
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
 # More models at https://huggingface.co/unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    #target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],

    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
from datasets import load_dataset, concatenate_datasets,Dataset
import pandas as pd

# 加载本地数据集
#dataset = load_dataset('json', data_files='/home/jovyan/train.jsonl', split='train')
# dataset = load_dataset("vicgalle/alpaca-gpt4", split = "train")
dataset1 = load_dataset('json', data_files='/home/jovyan/traindata/common_en_70k.jsonl', split='train')
print(len(dataset1))
dataset2 = load_dataset('json', data_files='/home/jovyan/traindata/common_zh_70k.jsonl', split='train')
print(len(dataset2))
dataset3 = load_dataset('json', data_files='/home/jovyan/traindata/computer_cn_26k_continue.jsonl', split='train')
print(len(dataset3))
dataset4 = load_dataset('json', data_files='/home/jovyan/traindata/computer_en_26k_continue.jsonl', split='train')
print(len(dataset4))
dataset6 = load_dataset('json', data_files='/home/jovyan/traindata/computer_en_26k(fixed).jsonl', split='train')
print(len(dataset6))
dataset5 = load_dataset('json', data_files='/home/jovyan/traindata/computer_zh_26k(fixed).jsonl', split='train')
print(len(dataset5))
dataset7 = load_dataset('json', data_files='/home/jovyan/traindata/unknow_zh_38k.jsonl', split='train')
print(len(dataset7))
dataset8 = load_dataset('json', data_files='/home/jovyan/traindata/unknow_zh_38k_continue.jsonl', split='train')
# 定义转换函数，将每个 conversation 转换为指定格式
combined_dataset = concatenate_datasets([dataset1, dataset2, dataset3,dataset4, dataset5,dataset6,dataset7, dataset8])

# 定义转换函数，将每个 conversation 转换为指定格式
def convert_conversation_format(conversation):
    new_format = {'conversations': []}

    # 遍历每个对话，将 human 和 assistant 成对地插入
    for convo in conversation:
        new_format['conversations'].append({'content': convo['human'], 'role': 'user'})
        new_format['conversations'].append({'content': convo['assistant'], 'role': 'assistant'})

    return new_format
# 遍历 dataset 并进行转换
converted_data = [convert_conversation_format(item['conversation']) for item in combined_dataset]

# 打印转换后的第一个结果

# Convert the list of converted data into a DataFrame
df = pd.DataFrame(converted_data)

# Create a Dataset from the DataFrame
converted_data_dataset = Dataset.from_pandas(df)

# Verify the dataset
print(converted_data_dataset[0])

In [ ]:
from datasets import load_dataset,concatenate_datasets
# 加载本地数据集
dataset = load_dataset('json', data_files='/home/jovyan/traindata/traintrain.jsonl', split='train')
## 数据集整合
from unsloth import to_sharegpt
dataset = to_sharegpt(
    dataset,
    merged_prompt = "{instruction}[[\n{input}]]",
    output_column_name = "output",
    conversation_extension = 4, # Select more to handle longer conversations
)
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
for _ in range(10):
    dataset = concatenate_datasets([dataset, dataset])
print(len(dataset))
dataset = concatenate_datasets([dataset,converted_data_dataset])

In [ ]:
chat_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{SYSTEM}<|eot_id|><|start_header_id|>user<|end_header_id|>

{INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{OUTPUT}<|eot_id|>"""
from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    #default_system_message = "你是中企动力的客服",
)
print(dataset[0])
print(len(dataset))


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 120,
        # num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                         # Change below!
    {"role": "user",      "content": "中企和中企动力是一家公司吗"}]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving
#model.save_pretrained_gguf("model", tokenizer, quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],)

# Save to 8bit Q8_0
#if False: model.save_pretrained_gguf("model", tokenizer,)
#if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")


In [ ]:
!cp /home/jovyan/traindata/Modelfile ./model/Modelfile

!cat ./model/Modelfile

In [ ]:
import subprocess
subprocess.Popen(["ollama", "serve"])
import time
time.sleep(3) # Wait for a few seconds for Ollama to load!
!ollama create wt1107 -f ./model/Modelfile
!kill -9 $(pgrep -f ollama)
print("结束")